In [1]:
# ! pip install emot
# ! pip install re

In [2]:
import pandas as pd
import emot, re
from sklearn.model_selection import train_test_split
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
pd.set_option("display.max_columns", 99)

In [3]:
! wget data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
! wget data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_2.csv
! wget data/full_dataset/ https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_3.csv
goemotions_1 = pd.read_csv('goemotions_1.csv')
goemotions_2 = pd.read_csv('goemotions_2.csv')
goemotions_3 = pd.read_csv('goemotions_3.csv')
frames = [goemotions_1, goemotions_2, goemotions_3]
df = pd.concat(frames, ignore_index=True)

--2022-03-13 14:36:28--  http://data/full_dataset/
Resolving data (data)... failed: nodename nor servname provided, or not known.
wget: unable to resolve host address ‘data’
--2022-03-13 14:36:29--  https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.160.112, 142.251.42.240, 172.217.160.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.160.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14174600 (14M) [application/octet-stream]
Saving to: ‘goemotions_1.csv.1’

goemotions_1.csv.1  100%[===================>]  13.52M  9.44MB/s    in 1.4s    

2022-03-13 14:36:31 (9.44 MB/s) - ‘goemotions_1.csv.1’ saved [14174600/14174600]

FINISHED --2022-03-13 14:36:31--
Total wall clock time: 2.6s
Downloaded: 1 files, 14M in 1.4s (9.44 MB/s)
--2022-03-13 14:36:31--  http://data/full_dataset/
Resolving data (data)... failed: nodename nor servnam

In [4]:
print(df.shape)
df.head(2)

(211225, 37)


,text,id,author,subreddit,link_id,parent_id,created_utc,rater_id,example_very_unclear,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,That game hurt.,eew5j0j,Brdd9,nrl,t3_ajis4z,t1_eew18eq,1.548381e+09,1,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,>sexuality shouldn’t be a grouping category I...,eemcysk,TheGreen888,unpopularopinion,t3_ai4q37,t3_ai4q37,1.548084e+09,37,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
df.example_very_unclear.value_counts()

False    207814
True       3411
Name: example_very_unclear, dtype: int64

In [6]:
%%time
# emotion label columns
lb_col = df.columns[9:]
print(f'Entire dataset: {df.shape}')
# remove examples that were very unclear. They do not have any emotions labels
df2 = df[~df['example_very_unclear']].copy()
print(f'{df2.shape[0]} rows left after removing records with unclear emotions')
# Sum the emotions together because there are different raters
df2 = df2[['text']+list(lb_col)].groupby(['text']).sum().reset_index()
print(f'{df2.shape[0]} rows left after deduping on text and id columns')
# keep records where there are at least two agreeing emotion labels
df2['at_least_2_agree'] = df2[lb_col].apply(lambda x: x.max(), axis=1)
df2 = df2[df2.at_least_2_agree >= 2]
df2 = df2.replace({1: 0, 2: 1, 3: 1, 4: 1, 5: 1}, inplace=False)
df_clean = df2.drop('at_least_2_agree', axis=1)
print(f'{df2.shape[0]} rows left after removing records without having at least two raters agreeing on the emotion')

Entire dataset: (211225, 37)
207814 rows left after removing records with unclear emotions
57730 rows left after deduping on text and id columns
53994 rows left after removing records without having at least two raters agreeing on the emotion
CPU times: user 3.96 s, sys: 267 ms, total: 4.23 s
Wall time: 4.45 s


In [7]:
# A surprise to be sure, but a welcome one.
# a = pd.read_csv("train.tsv",sep='\t',names=['a','b']).reset_index().rename(columns={'index':'text'})
# a[a.text.str.contains('A surprise to be sure, but a welcome one.')]

In [8]:
print(df_clean.shape)
df_clean.head(2)

(53994, 29)


,text,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,"""If you don't wear BROWN AND ORANGE...YOU DON...",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,"""What do Scottish people look like?"" How I wo...",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [9]:
def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        if emot in text:
            orig = text
            text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
            print(f'{orig} -> {text}', '\n')
    return text

def convert_emoticons(text):
    for emot in EMOTICONS_EMO:
        if emot in text:
            orig = text
            text = text.replace(emot, "_".join(EMOTICONS_EMO[emot].split()))
            print(f'{orig} -> {text}', '\n')
    return text

In [10]:
%%time
print('-------replacing emoji with text---------')
df_clean['text'] = df_clean['text'].apply(lambda x: convert_emojis(x))
print('-------replacing emoticon with text---------')
df_clean['text'] = df_clean['text'].apply(lambda x: convert_emoticons(x))

-------replacing emoji with text---------
 I'm scared to even ask my mom ,I might get yelled at 😟 ->  I'm scared to even ask my mom ,I might get yelled at worried_face 

 🎶 Hark to the tale of [NAME], and the boy he loved so dear. They remained the best of friends for years and years and years. 🎶  ->  musical_notes Hark to the tale of [NAME], and the boy he loved so dear. They remained the best of friends for years and years and years. musical_notes  

"I awoke several hours later in a daze...." 😂 -> "I awoke several hours later in a daze...." face_with_tears_of_joy 

#FURRIES 😾 OUT, OUT! ^👉 #🤚 WE NEED A WALL 🖐️ AROUND R/SUBREDDITDRAMA 👋 AND MAKE R/FURRY_IRL 👐 PAY FOR IT!! ☝️ -> #FURRIES 😾 OUT, OUT! ^backhand_index_pointing_right #🤚 WE NEED A WALL 🖐️ AROUND R/SUBREDDITDRAMA 👋 AND MAKE R/FURRY_IRL 👐 PAY FOR IT!! ☝️ 

#FURRIES 😾 OUT, OUT! ^backhand_index_pointing_right #🤚 WE NEED A WALL 🖐️ AROUND R/SUBREDDITDRAMA 👋 AND MAKE R/FURRY_IRL 👐 PAY FOR IT!! ☝️ -> #FURRIES 😾 OUT, OUT! ^backhand_

Any response helps and makes me feel like I’m not alone....thank you❤️ -> Any response helps and makes me feel like I’m not alone....thank youred_heart️ 

Apparently my state 😔 -> Apparently my state pensive_face 

Are you in Twitter with a similar s/n? I feel like I remember this name when I’ve been posting about teen mom 😂 -> Are you in Twitter with a similar s/n? I feel like I remember this name when I’ve been posting about teen mom face_with_tears_of_joy 

Are you one of them? Sorry. You asked for it. 😂 -> Are you one of them? Sorry. You asked for it. face_with_tears_of_joy 

Aren’t people on the internet the best! I don’t blame you for deleting, no time for negativity in this life. 🍻  -> Aren’t people on the internet the best! I don’t blame you for deleting, no time for negativity in this life. clinking_beer_mugs  

Arm sleeve DSJ🙌🏻🙌🏻🙌🏻 -> Arm sleeve DSJ🙌light_skin_tone🙌light_skin_tone🙌light_skin_tone 

Arm sleeve DSJ🙌light_skin_tone🙌light_skin_tone🙌light_skin_tone -> Arm sleeve D

Calling Hollywood...would make a great movie.☺ -> Calling Hollywood...would make a great movie.smiling_face 

Came here to say this. I thank you kindly. Long live America. 😁 -> Came here to say this. I thank you kindly. Long live America. beaming_face_with_smiling_eyes 

Can a gang member legally citizen arrest someone 🤔 -> Can a gang member legally citizen arrest someone thinking_face 

Can't relate because the sadness and loneliness is constant 😎 -> Can't relate because the sadness and loneliness is constant smiling_face_with_sunglasses 

Can't wait to see this badboy in season 8 😍 😍 😍 When [NAME] listed him among Dragonpit characters i was so happy.  -> Can't wait to see this badboy in season 8 smiling_face_with_heart-eyes smiling_face_with_heart-eyes smiling_face_with_heart-eyes When [NAME] listed him among Dragonpit characters i was so happy.  

Can’t wait for the saas drama!!! 😂😂 -> Can’t wait for the saas drama!!! face_with_tears_of_joyface_with_tears_of_joy 

Capitalism coerces

Eff your video - love Canada 🇨🇦 Stupid geolock -> Eff your video - love Canada Canada Stupid geolock 

Eh, some people grow up, get tired of banging their heads and decide to start using them instead 😜 -> Eh, some people grow up, get tired of banging their heads and decide to start using them instead winking_face_with_tongue 

Enchanted - [NAME]😍 -> Enchanted - [NAME]smiling_face_with_heart-eyes 

Even after reading all your posts, I still don't understand... 😑 -> Even after reading all your posts, I still don't understand... expressionless_face 

Exactly hate when people try to spit facts when they don’t know if those facts are even right😂😂 -> Exactly hate when people try to spit facts when they don’t know if those facts are even rightface_with_tears_of_joyface_with_tears_of_joy 

Excellent collage, I learned so much from all these guys. The community is great, the unconfiscatable conference was awesome 🙏🏽🙏🏽 -> Excellent collage, I learned so much from all these guys. The community is

Happy Cake Day [NAME]! Enjoy this slice on me 🍰 -> Happy Cake Day [NAME]! Enjoy this slice on me shortcake 

Happy Cakeday! I met my partner of 6 years at a London [NAME] meet-up 🤩 -> Happy Cakeday! I met my partner of 6 years at a London [NAME] meet-up star-struck 

Happy New Year you wonderful mummy you! I was thinking about you today and hoping you were doing okay ☺ -> Happy New Year you wonderful mummy you! I was thinking about you today and hoping you were doing okay smiling_face 

Happy New year! 🤗 -> Happy New year! hugging_face 

Happy anniversary!! ❤ -> Happy anniversary!! red_heart 

Happy birthday my guy! Hope you have a great day today, tomorrow, and every day after that. You and everyone deserve a good day. 😃 -> Happy birthday my guy! Hope you have a great day today, tomorrow, and every day after that. You and everyone deserve a good day. grinning_face_with_big_eyes 

Happy birthday 🎂!! -> Happy birthday birthday_cake!! 

Happy birthday!! Lots of love and hugs from a stran

...

Thank you kindly :) -> Thank you kindly Happy_face_or_smiley 

Thank you so much :) -> Thank you so much Happy_face_or_smiley 

Thank you so much for posting this! I've thought this way for a long time and it's very comforting to see that I'm not alone :3 -> Thank you so much for posting this! I've thought this way for a long time and it's very comforting to see that I'm not alone Happy_face_smiley 

Thank you so much for this advice :) i'm just really worried about going through the process again -> Thank you so much for this advice Happy_face_or_smiley i'm just really worried about going through the process again 

Thank you so much! We will likely also be doing that haha :) -> Thank you so much! We will likely also be doing that haha Happy_face_or_smiley 

Thank you! :) -> Thank you! Happy_face_or_smiley 

Thank you! :O -> Thank you! Surprise 

Thank you! I agree. We’ve decided to back off. No need in feeding into it. :-) -> Thank you! I agree. We’ve decided to back off. No need in 

That’s awesome! Good call :) settling my alarm now m/ -> That’s awesome! Good call Happy_face_or_smiley settling my alarm now m/ 

That’s better:-) now go get em tiger -> That’s betterHappy_face_smiley now go get em tiger 

That’s cool. I didn’t know they had cameras like that. Thank you for sharing :) -> That’s cool. I didn’t know they had cameras like that. Thank you for sharing Happy_face_or_smiley 

That’s great of you :) -> That’s great of you Happy_face_or_smiley 

That’s my mission :) thank you so much -> That’s my mission Happy_face_or_smiley thank you so much 

That’s one of the many sleeping cycles of my fucked-up schedule :) -> That’s one of the many sleeping cycles of my fucked-up schedule Happy_face_or_smiley 

That’s pretty awesome :) -> That’s pretty awesome Happy_face_or_smiley 

That’s what we keep trying to tell TSCC!!! ;) -> That’s what we keep trying to tell TSCC!!! Wink_or_smirk 

That’s wonderful to hear. :) I hope you get your guy! -> That’s wonderful to hear. Ha

Was not one of those players, still want to go back to the MW2 days. Totally would if I still had my PS3 :/ -> Was not one of those players, still want to go back to the MW2 days. Totally would if I still had my PS3 Skeptical,_annoyed,_undecided,_uneasy_or_hesitant 

Was your father a member? Or did you save a member? :o (very cool!) -> Was your father a member? Or did you save a member? Surprise (very cool!) 

We all were. Because we’re the responsible people watching the show, and not the glorious train wrecks starring in it :) -> We all were. Because we’re the responsible people watching the show, and not the glorious train wrecks starring in it Happy_face_or_smiley 

We also thought about that :)! I'll try look for other options! -> We also thought about that Happy_face_or_smiley! I'll try look for other options! 

We don't deserve you. Thanks for the update. Happy it's being handled! :) -> We don't deserve you. Thanks for the update. Happy it's being handled! Happy_face_or_smiley 

You had a tough childhood :( -> You had a tough childhood Frown,_sad,_andry_or_pouting 

You have the bear mask?? :D it is glorious! -> You have the bear mask?? Laughing,_big_grin_or_laugh_with_glasses it is glorious! 

You know, when I think of democracy, the first thing that comes to mind is best 2:3... Errr... 3:5.….. uhmm... I mean 5:7... /**S** -> You know, when I think of democracy, the first thing that comes to mind is best 2Happy_face_smiley... Errr... 3:5.….. uhmm... I mean 5:7... /**S** 

You look adorable. :) I love your eyes. :) -> You look adorable. Happy_face_or_smiley I love your eyes. Happy_face_or_smiley 

You look like [NAME] from firefly :) -> You look like [NAME] from firefly Happy_face_or_smiley 

You lost everything. So sad. Lots of lies from you. Next time do better. I love this. Friend. :) -> You lost everything. So sad. Lots of lies from you. Next time do better. I love this. Friend. Happy_face_or_smiley 

You mean FUStrated, it's [NAME] here :) -> You mean FUS

yeah certainly shouldn't feel this way but i do now :/ -> yeah certainly shouldn't feel this way but i do now Skeptical,_annoyed,_undecided,_uneasy_or_hesitant 

yeah i want to see what kind of car it is post pics :) -> yeah i want to see what kind of car it is post pics Happy_face_or_smiley 

yeah, [NAME] is who i was talking about :( love her but soon as she starts talking about baby stuff, i'm out -> yeah, [NAME] is who i was talking about Frown,_sad,_andry_or_pouting love her but soon as she starts talking about baby stuff, i'm out 

yes but how many new characters were there? Checkmate atheists. :) -> yes but how many new characters were there? Checkmate atheists. Happy_face_or_smiley 

yes, please do I'm worried about her D: -> yes, please do I'm worried about her Sadness 

yes. to the above number i provided :) hope it’s helpful! -> yes. to the above number i provided Happy_face_or_smiley hope it’s helpful! 

yes:) -> yesHappy_face_or_smiley 

you got me for a sec. I was like wo

In [11]:
X = df_clean['text']
y = df_clean.iloc[:,2:]
y.head(2)

,amusement,anger,annoyance,approval,caring,confusion,curiosity,desire,disappointment,disapproval,disgust,embarrassment,excitement,fear,gratitude,grief,joy,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=2)
print(f'X_train shape: {X_train.shape}')
print(f'X_val shape: {X_val.shape}')
print(f'X_test shape: {X_test.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_val shape: {y_val.shape}')
print(f'y_test shape: {y_test.shape}')

X_train shape: (43195,)
X_val shape: (5399,)
X_test shape: (5400,)
y_train shape: (43195, 27)
y_val shape: (5399, 27)
y_test shape: (5400, 27)
